# Setup

In [1]:
import camelot
import pandas as pd

We need to process each PDF manually, as the table format does not stay the same and the extraction does not work perfectly. So the below code needs tweaking in some cases (rows/columns to drop, ...).

In [2]:
date = '2021-05-12'

In [3]:
file = 'https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/DESH/Bericht_VOC_{}.pdf?__blob=publicationFile'.format(date)

# Read PDF

Here, we manually select the page that contains the table of interest.

In [4]:
tables = camelot.read_pdf(file, pages = "7", flavor="stream", row_tol=5)

In [5]:
tables[0].df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,,"und B.1.617.2, in der Menge der zufällig für d...",,,,,,,,,,
1,2021 aufgeteilt nach Kalenderwochen.,,,,,,,,,,,
2,,B.1.1.7,,B.1.351,,,B.1.617.1,,B.1.617.2,,P.1,
3,,Anzahl,Anteil,Anzahl,Anteil,Anzahl,Anteil,Anzahl,Anteil,Anzahl,,Anteil
4,KW 2021,,(%),,(%),,(%),,(%),,,(%)
5,01-17,31.125,"59,1",621,"1,2",51,"0,1",39,"0,1",87,,"0,2"
6,01,4,2,0,0,0,0,0,0,0,,0
7,02,47,"8,3",2,"0,4",0,0,0,0,0,,0
8,03,79,"4,7",11,"0,7",0,0,0,0,0,,0
9,04,273,"10,2",12,"0,4",0,0,0,0,0,,0


# Data preparation

Sometimes the beginning of the table is not perfectly detected. We drop unnecessary leading rows.

In [6]:
df = tables[0].df.iloc[2:]

We detect all mentioned variants and create new column names from these.

In [7]:
variants = [v for v in df.iloc[0].values if v != '']
cols = [v + a for v in variants for a in ['_count', '_proportion']]
cols = ['week'] + cols

In [8]:
cols

['week',
 'B.1.1.7_count',
 'B.1.1.7_proportion',
 'B.1.351_count',
 'B.1.351_proportion',
 'B.1.617.1_count',
 'B.1.617.1_proportion',
 'B.1.617.2_count',
 'B.1.617.2_proportion',
 'P.1_count',
 'P.1_proportion']

Sometimes too many columns are detected, we drop these

In [9]:
df = df.drop(columns=[10])

Rename columns

In [10]:
df.columns = cols

In [11]:
df.head()

,week,B.1.1.7_count,B.1.1.7_proportion,B.1.351_count,B.1.351_proportion,B.1.617.1_count,B.1.617.1_proportion,B.1.617.2_count,B.1.617.2_proportion,P.1_count,P.1_proportion
2,,B.1.1.7,,B.1.351,,,B.1.617.1,,B.1.617.2,,
3,,Anzahl,Anteil,Anzahl,Anteil,Anzahl,Anteil,Anzahl,Anteil,Anzahl,Anteil
4,KW 2021,,(%),,(%),,(%),,(%),,(%)
5,01-17,31.125,"59,1",621,"1,2",51,"0,1",39,"0,1",87,"0,2"
6,01,4,2,0,0,0,0,0,0,0,0


Cut beginning of table and start with first row that contains the relevant data

In [12]:
df = df.iloc[4:, ].reset_index(drop=True)

Change formatting

In [13]:
df.replace({'\.': '', ',': '.'}, regex=True, inplace=True)

In [14]:
df.tail()

,week,B.1.1.7_count,B.1.1.7_proportion,B.1.351_count,B.1.351_proportion,B.1.617.1_count,B.1.617.1_proportion,B.1.617.2_count,B.1.617.2_proportion,P.1_count,P.1_proportion
12,13,3263,87,49,1.3,0,0,0,0,8,0.2
13,14,3329,88.5,74,2,3,0.1,1,0,10,0.3
14,15,3652,88.9,83,2,8,0.2,3,0.1,11,0.3
15,16,3869,91.1,41,1,18,0.4,15,0.4,14,0.3
16,17,1860,91.7,12,0.6,12,0.6,19,0.9,10,0.5


# Export

In [15]:
df.to_csv('../data/archive/{}_variants_of_concern_sample.csv'.format(date), index=False)